# Diagnostic Measures
Now that we have established both the core assumptions of the linear model and have discussed some important data features, we can now examine some of the key diagnostic measures we will use for assessing both of these elements. Importantly, these diagnostics are all calculated directly from the model itself. As such, we do not usually check the assumptions *before* fitting the model. Instead, we fit the model, derive diagnostic measures from the model and then check its suitability. This is important because we want our diagnostics to be tied as closely to the model as possible. Although possible to create useful visualisations from the raw data, we can only accurately assess the model if we have the precise numeric values the model is using. This necessitates fitting the model *first*. However, we need to take care that we do not enter into *interpreting* the model until we are satisfied the assumptions are suitably met.

## Leverage Values
We will start our tour of diagnostic measures with the *leverage values* because, as we will see below, these have some important consequences for interpreting other diagnostic measures. As mentioned above, leverage concerns the degree to which a single data point is influencing the model fit. The *leverage values* are measures of leverage and range $0 \leq h_{i} \leq 1$. These can be interpreted as

- If $h_{i}$ is close to 0, then the predicted value $\hat{y}_{i}$ is mostly determined by the other data points.
- If $h_{i}$ is close to 1, then the predicted value $\hat{y}_{i}$ is determined almost entirely by $y_{i}$. 

So, in this sense, higher leverage means that the predicted value of point $i$ is not very related to the rest of the data in the dataset, as it depends almost entirely on that single point. This implies that this point is an outlier in predictor space, but also that the model fit for that combination of predictor values is not a balance between mutliple observations, as it is biased towards that single observation.

The actual calculation of the leverage values in multiple regression is complex and involves diving into the linear algebraic representation of linear models (which is beyond the scope of the unit). However, they can be easily extracted in `R` using the `hatvalues()` function[^hatmat-foot]

In [1]:
data(mtcars)
mod <- lm(mpg ~ wt + hp + cyl, data=mtcars)
lev <- hatvalues(mod)
print(lev)

          Mazda RX4       Mazda RX4 Wag          Datsun 710      Hornet 4 Drive 
         0.07495754          0.05815750          0.08563345          0.05334688 
  Hornet Sportabout             Valiant          Duster 360           Merc 240D 
         0.10981841          0.06986080          0.11762078          0.15741830 
           Merc 230            Merc 280           Merc 280C          Merc 450SE 
         0.15201954          0.04691578          0.04691578          0.07862927 
         Merc 450SL         Merc 450SLC  Cadillac Fleetwood Lincoln Continental 
         0.08370579          0.08169851          0.20155034          0.24373270 
  Chrysler Imperial            Fiat 128         Honda Civic      Toyota Corolla 
         0.23547715          0.08274176          0.13078212          0.09961207 
      Toyota Corona    Dodge Challenger         AMC Javelin          Camaro Z28 
         0.09155927          0.14918089          0.15654328          0.10338967 
   Pontiac Firebird         

None of the data here exerts very large leverage in *absolute* terms, as all the leverage values are quite far below 1. Neverthless, it is typical to interpret leverage in a *relative* fashion for a given dataset. As such, a common cutoff for defining *high leverage* is $h_{i} > 2\frac{p}{n}$, where $p$ is the number of model parameters and $n$ is the sample size. For reasons related to how leverage is calculated, this gives a cutoff of *twice* the average leverage[^levcutoff-foot]. For the `mtcars` model above this would be

In [2]:
n       <- length(mod$fitted.values)
p       <- length(mod$coefficients)
big.lev <- 2*p/n
print(big.lev)

[1] 0.25


So our *relative* cutoff would be $h_{i} > 0.25$. We can examine any concerning cases using

In [3]:
print(lev[lev > big.lev])

Maserati Bora 
    0.4635658 


So we have one observation with high relative leverage, suggesting this may be an outlier in predictor space and may be unduly influencing the model fit. We would need to examine this specific case in more detail and assess why its particular combination of predictor values is unusual, and whether any mistakes have been made. We could also assess the sensitivity of our model to this point by fitting it both *with* and *without* this observation, taking note of whether our inference substantially changes. We will see how leverage factors into some standard diagnostic plots a little later.

## Residuals
Beyond leverage values, one of the most useful diagnostic we can get from our model is the *residuals*. As we know, many of the linear model assumptions centre on the distribution of the errors. As such, it would stand to reason that we can use the residuals as a proxy for the errors to assess these assumptions (though there is a catch, as we will discuss below). Beyond the distributional assumptions, we can also use the residuals to detect outliers, check for evidence of non-linearity in the model fit, and can diagnose any issues with the assumption of a continuous outcome when working with data that is technically non-continuous. So, as we can see, the residuals are incredibly useful as a diagnostic tool and can be extracted using the `resid()` function in `R`.

In [4]:
resid.raw <- resid(mod)
print(resid.raw)

          Mazda RX4       Mazda RX4 Wag          Datsun 710      Hornet 4 Drive 
         -1.8204257          -1.0128476          -3.1603990           0.4639233 
  Hornet Sportabout             Valiant          Duster 360           Merc 240D 
          1.5322025          -2.1503588          -1.1934238           0.6356864 
           Merc 230            Merc 280           Merc 280C          Merc 450SE 
         -0.4957351          -0.7890124          -2.1890124           1.3175861 
         Merc 450SL         Merc 450SLC  Cadillac Fleetwood Lincoln Continental 
          1.1408152          -0.8008361          -0.4944331           0.2370012 
  Chrysler Imperial            Fiat 128         Honda Civic      Toyota Corolla 
          4.5573819           5.5725355           1.4673228           5.8985522 
      Toyota Corona    Dodge Challenger         AMC Javelin          Camaro Z28 
         -3.9290355          -1.8653922          -2.4345849          -1.3383411 
   Pontiac Firebird         

For each car, the residual value represents the difference between the model prediction and the original data. The residuals are therefore in the original units of the outcome which, in this example, is MPG. This can be difficult to interpret in terms of defining which residuals are *large*, but this is something we will come back to below.

### Some Complexities with Raw Residuals
As indicated above, it would seem reasonable to assume that we can use the residuals as a proxy for the true errors and thus use them to assess the distributional assumptions of our model. Although we can do this in an *approximate* fashion, there is an important technical detail to consider. One of the main reasons that we have made a point of distinguishing *errors* from *residuals* is that the estimation process *changes the distributional properties of the errors*. This means that *errors* and *residuals* are not expected to behave identically. So while it is correct to assume

$$
\epsilon_{i} \overset{\text{i.i.d.}}{\sim} \mathcal{N}\left(0,\sigma^{2}\right),
$$

it is *not* technically correct to assume the same for the *residuals*. In fact, the variance of the residuals is *not* $\sigma^{2}$, as we might expect, it is

$$
\text{Var}\left(e_{i}\right) = \sigma^{2}\left(1 - h_{i}\right).
$$

So, rather than being constant (as is the case with the true *errors*), the variance of the residuals is *scaled* by the leverage. Because we have a leverage value for each observation, the variance will be different for each observation. In other words, the residuals *cannot be homoscedastic*, even if the errors are. In the equation above, the key element is that the variance is scaled by $\left(1 - h_{i}\right)$. So, if $h_{i} = 0.8$ then the variance would be multiplied by $1 - 0.8 = 0.2$. As such, the *higher* the leverage, the *smaller* the variance gets.

```{admonition} Why Does Leverage Reduce the Variance?
:class: tip
The reason that this relationship between leverage and variance exists is because of the inherent properties of the estimation process. This is easiest to think about in terms of *least-squares*. In order to minimise the sum of squared residuals, observations with high leverage pull the model fit towards them. Because there may be no other points around them, there is nothing to balance the model fit by pulling in the opposite direction. The result is that the variance around those points will be smaller than elsewhere in the data, where the fit is more of a balanced between many data points. As such, points with high leverage will often sit closer to the regression line, making the variance *smaller* for that particular combination of predictor values. The question is then whether the influence of these individual data points is biasing the model, despite initial appearences of a good fit.
```

### Standardised Residuals
At this point, we have established that residuals are potentially very useful diagnostically, but they have two issues:

1. Residuals are in the original units of the outcome and therefore difficult to interpret in terms of which are *large* and which are *small*.
2. The homoscedasticity of the residuals depends upon the leverage, even if the true errors are homoscedastic. So any assessment of constant variance that uses the residuals could be misleading.

In order to solve both these issues, we can *standardise* the residuals by computing 

$$
r_{i} = \frac{e_{i}}{\hat{\sigma}\sqrt{1 - h_{i}}},
$$

which is just the residual divided by its theoretical standard deviation. This has the effect of scaling the residuals into units of standard deviation. So, for the purpose of outlier detection, we just need to decide how many standard deviations we would consider *large*. Typically, values of either 2 or 3 standard deviations are used to detect outliers. This scaling also has an additional advantage because the effect of leverage is removed from each residual. This makes standardised residuals useful for decting outliers *and* for assessing homoscedasticity, as the misleading effect of leverage is gone. Standardised residuals can be calculated in `R` using the `rstandard()` function.

In [5]:
data(mtcars)
mod       <- lm(mpg ~ wt + hp + cyl, data=mtcars)
resid.std <- rstandard(mod)
print(resid.std)

          Mazda RX4       Mazda RX4 Wag          Datsun 710      Hornet 4 Drive 
         -0.7536168          -0.4155405          -1.3159524           0.1898494 
  Hornet Sportabout             Valiant          Duster 360           Merc 240D 
          0.6465994          -0.8877596          -0.5058544           0.2757372 
           Merc 230            Merc 280           Merc 280C          Merc 450SE 
         -0.2143459          -0.3217930          -0.8927730           0.5465378 
         Merc 450SL         Merc 450SLC  Cadillac Fleetwood Lincoln Continental 
          0.4745219          -0.3327434          -0.2203141           0.1085104 
  Chrysler Imperial            Fiat 128         Honda Civic      Toyota Corolla 
          2.0752892           2.3166769           0.6266424           2.4750788 
      Toyota Corona    Dodge Challenger         AMC Javelin          Camaro Z28 
         -1.6413307          -0.8052115          -1.0554848          -0.5627602 
   Pontiac Firebird         

Here we can see a few observations with residuals larger than 2 standard deviations from the model fit.

In [6]:
print(resid.std[abs(resid.std) > 2])

Chrysler Imperial          Fiat 128    Toyota Corolla 
         2.075289          2.316677          2.475079 


These could be potential outliers, though we will see ways of visualising these later so that they can be seen in context with the rest of the data. We will also see how visualisations can be used to assess homoscedasticity.

### Studentised Residuals
An alternative, but potentially more powerful approach, to standardised residuals is to computer *studentised* residuals. These are based on scaling the residuals into standard deviation units, but they do so differently for each individual datapoint. Rather than using the global variance estimate $\hat{\sigma}^{2}$, studentised residuals use $\hat{\sigma}^{2}_{-i}$, which is the variance estimate from a model with datapoint $i$ removed. 

$$
t_{i} = \frac{e_{i}}{\hat{\sigma}_{-i}\sqrt{1 - h_{i}}},
$$

The reason for doing this is that an outlier will artifically inflate the global variance estimate, meaning that extreme points will have *smaller* values when standardised. In other words, outliers can *hide* and appear less extreme than they really are. This is corrected by removing the point in question from the variance when scaling it into standard deviation units. Studentised residuals can be calculated using the `rstudent()` function in `R`.

In [7]:
resid.t <- rstudent(mod)
print(resid.t)

          Mazda RX4       Mazda RX4 Wag          Datsun 710      Hornet 4 Drive 
         -0.7476584          -0.4093168          -1.3341552           0.1865485 
  Hornet Sportabout             Valiant          Duster 360           Merc 240D 
          0.6397422          -0.8842967          -0.4990246           0.2711369 
           Merc 230            Merc 280           Merc 280C          Merc 450SE 
         -0.2106564          -0.3165804          -0.8894363           0.5395753 
         Merc 450SL         Merc 450SLC  Cadillac Fleetwood Lincoln Continental 
          0.4678563          -0.3273955          -0.2165319           0.1065775 
  Chrysler Imperial            Fiat 128         Honda Civic      Toyota Corolla 
          2.2153833           2.5303244           0.6197115           2.7498370 
      Toyota Corona    Dodge Challenger         AMC Javelin          Camaro Z28 
         -1.6953756          -0.8000188          -1.0577210          -0.5557716 
   Pontiac Firebird         

Here we can see that the previous standardised residuals with values over 2 have all gotten *bigger* when using studentised residuals. This shows how their inclusion was artifically inflating the variance and thus making their standardised score *smaller*. This makes studentised residuals much more useful for detecting outliers.

In [8]:
print(resid.t[abs(resid.t) > 2])

Chrysler Imperial          Fiat 128    Toyota Corolla 
         2.215383          2.530324          2.749837 


It is also interesting to note that the `Maserati Bora`, which had high relative leverage, has *not* been identified as an outlier. This highlights how the two concepts are somewhat independent, particularly when we recognise that observations with high leverage will often live close to the regression line and thus will have *smaller* residuals.

```{admonition} The Distribution of Studentised Residuals
:class: tip
Another advantage of *studentised* residuals is that they have a known distribution. As the notation $t_{i}$ implies, the residuals are distributed as a $t$-distribution with $n-p-1$ degrees of freedom. This means that we *could* calculate a $p$-value for each residual to test the null hypothesis that this observation is consistent with the model. For instance, we can calculate `2 * pt(abs(resid.t), df=mod$df.residual-1, lower.tail=FALSE)` to get $p$-values for the studentised residuals. *However*, we have calculated $n$ tests here and so would need some form of correction (which we will discuss further next week). Furthermore, we are again relying on NHST here for assumption checking, which we have already indicated is flawed. However, we will see how this distribution crops up again a little later on. 
```

## Predicted Values
Beyond the residuals and their standardisation, the *predicted* values from the model are very useful diagnostically. As a reminder, the predicted values in a regression model are

$$
\hat{y}_{i} = E(y_{i}|\mathbf{x}_{i}) = \hat{\beta}_{0} + \sum_{j=1}^{p} \hat{\beta}_{j}x_{ij},
$$

which are the results of evaluating the model equation for each observation. These are also known as the *fitted* values, and can be extracted using the `fitted()` function in `R`.

In [9]:
mod.fit <- fitted(mod)
print(mod.fit)

          Mazda RX4       Mazda RX4 Wag          Datsun 710      Hornet 4 Drive 
           22.82043            22.01285            25.96040            20.93608 
  Hornet Sportabout             Valiant          Duster 360           Merc 240D 
           17.16780            20.25036            15.49342            23.76431 
           Merc 230            Merc 280           Merc 280C          Merc 450SE 
           23.29574            19.98901            19.98901            15.08241 
         Merc 450SL         Merc 450SLC  Cadillac Fleetwood Lincoln Continental 
           16.15918            16.00084            10.89443            10.16300 
  Chrysler Imperial            Fiat 128         Honda Civic      Toyota Corolla 
           10.14262            26.82746            28.93268            28.00145 
      Toyota Corona    Dodge Challenger         AMC Javelin          Camaro Z28 
           25.42904            17.36539            17.63458            14.63834 
   Pontiac Firebird         

The reason why the prediction is useful diagnostically is because our assumptions about homoscedasticity relate to an equal spread of data above and below the regression plane. Each predicted value represents a point on this plane from some combination of predictor values. As such, when we plot the predicted values against the standardised residuals, we should see an even and equal vertical spread of points above and below the predictions. We can also use this form of visualisation to assess evidence of *non-linear* relationships in the data. For instance, if we have a straight-line fit but the actual relationship is a *curve*, the residuals will not be evenly spread around the prediction and will have some characteristic shape that bends around the prediction. We will see this when we examine some standard diagnostic plots in the next part of the lesson.

## Cook's Distance
Another common diagnostic measure is a quantity known as *Cook's Distance*, which combines both the studentised residuals and the leverage to produce a single metric of whether an observation is an outlier in *both* outcome and predictor space. For each observation, the Cook's Distance $D_{i}$ is calculated as

$$
D_{i} = \frac{1}{p} \times t_{i}^{2} \times \frac{h_{i}}{1 - h_{i}},
$$

These values can be returned in `R` using the `cooks.distance()` function

In [10]:
D <- cooks.distance(mod)
print(D)

          Mazda RX4       Mazda RX4 Wag          Datsun 710      Hornet 4 Drive 
       0.0115052172        0.0026655956        0.0405455760        0.0005077811 
  Hornet Sportabout             Valiant          Duster 360           Merc 240D 
       0.0128945786        0.0147984549        0.0085274628        0.0035511903 
           Merc 230            Merc 280           Merc 280C          Merc 450SE 
       0.0020591309        0.0012743271        0.0098086603        0.0063727998 
         Merc 450SL         Merc 450SLC  Cadillac Fleetwood Lincoln Continental 
       0.0051424860        0.0024625653        0.0030630946        0.0009486833 
  Chrysler Imperial            Fiat 128         Honda Civic      Toyota Corolla 
       0.3316313326        0.1210330843        0.0147706379        0.1694339333 
      Toyota Corona    Dodge Challenger         AMC Javelin          Camaro Z28 
       0.0678793903        0.0284207738        0.0516910757        0.0091297778 
   Pontiac Firebird         

In terms of interpretation, values of $D_{i} > 0.5$ are cause for concern and values of $D_{i} > 1$ are indications of a potential problem. Remember, this relates to a data point being extreme relative to the model fit *and* in terms of the patterns of predictors. This does not necessarily make it *wrong*, but does require investigation. We can filter the returned values of $D_{i}$ to check

In [11]:
print(D[D > 0.5])
print(D[D > 1])

named numeric(0)
named numeric(0)


In this case, we have *no* observations with high values of $D_{i}$.

## The Variance Inflation Factor (VIF)
Our final diagnostic measure deviates from the other measures discussed in this section because it is only a single number, is not a direct output from the model and requires an external package to calculate. Neverthless, this is one of the most useful tools because it helps diagnose issues of *multicollinearity*. In the previous section, we discussed multicollinearity in terms of high correlation between predictors. As a diagnostic measure, we can certainly calculate the correlation amongst predictors and use that to highlight potential problems. For instance, we can calculate the correlation between the predictors easily in `R` using

In [14]:
set.seed(666)
data(mtcars)
wt      <- mtcars$wt
wt.copy <- wt + rnorm(n=length(wt), mean=0, sd=0.2)
preds   <- cbind(mtcars,wt.copy)[,c('wt','wt.copy','hp','cyl')] # just the predictors

print(cor(preds))

               wt   wt.copy        hp       cyl
wt      1.0000000 0.9734991 0.6587479 0.7824958
wt.copy 0.9734991 1.0000000 0.6101981 0.7534608
hp      0.6587479 0.6101981 1.0000000 0.8324475
cyl     0.7824958 0.7534608 0.8324475 1.0000000


A threshold of $r > 0.8$ is sometimes used for this purpose, though lower threshold of $r > 0.7$ or $r > 0.6$ are sometimes seen. Depending on the choice, there is either a concern only for `wt.copy`, or a concern for *all* the predictors in this model. This indicates that there is a strong degree of interdependency between all these measurements, which should give us pause to consider whether any of our predictors are redundant or even whether our questions about these data are sensible. In addition, this highlight an issue with raw correlation because this is only an indictor that there *might* be a problem. For the purpose of characterising whether there actually *is* a problem, a better metric is the *Variance Inflation Factor* (VIF).  

Remembering back to when we defined the sampling distribution of the coefficients in a multiple regression model, we stated that

$$
\hat{\beta}_{j} \sim \mathcal{N}\left(\beta_{j},\frac{\sigma^{2}}{\sum{\left(x_{ij} - \bar{x}_{j}\right)^{2} \times \left(1 - R^{2}_{j}\right)}}\right).
$$

From this, we can see that the variance is

$$
\text{Var}\left(\hat{\beta}_{j}\right) = \frac{\sigma^{2}}{\sum{\left(x_{ij} - \bar{x}_{j}\right)^{2} \times \left(1 - R^{2}_{j}\right)}}
$$

which we can rewrite as

$$
\text{Var}\left(\hat{\beta}_{j}\right) = \frac{\sigma^{2}}{\sum{\left(x_{ij} - \bar{x}_{j}\right)^{2}}} \times \frac{1}{1 - R^{2}_{j}}
$$

So the variance can be taken as the expression we already know from simple regression, scaled by the factor $\frac{1}{1 - R^{2}}$. This is precisely the *variance inflation factor*. So

$$
\text{VIF}_{j} = \frac{1}{1 - R^{2}_{j}}
$$

and the variance of our parameter estimate can be expressed as

$$
\text{Var}\left(\hat{\beta}_{j}\right) = \frac{\sigma^{2}}{\sum{\left(x_{ij} - \bar{x}_{j}\right)^{2}}} \times \text{VIF}_{j}
$$

The term $R^{2}_{j}$ is something we will cover in more detail *next week* when we discuss model comparisons. For the moment, this can be understood as the proportion of total variation in predictor $j$ that can be explained by all the other predictors in the model. So, when predictors are independent, $R^{2}_{j} = 0$ and $\text{VIF}_{j} = \frac{1}{1 - 0} = 1$. This means that there is *no scaling* and the variance of $\hat{\beta}_{j}$ is the same as in simple regression. However, for non-independent predictors with (for example) $R^{2}_{j} = 0.9$, we would have $\text{VIF}_{j} = \frac{1}{1 - 0.9} = \frac{1}{0.1} = 10$. This means that the variance of $\hat{\beta}_{j}$ is *ten times larger* than it would be if there were independence. 

In terms of calculating the VIF, we can do this easily using the `vif()` function from the `car` package.

In [15]:
library(car)
mod.multicol <- lm(mpg ~ wt + wt.copy + hp + cyl, data=mtcars)
print(vif(mod.multicol))

Loading required package: carData



       wt   wt.copy        hp       cyl 
22.174855 19.921678  3.383464  4.794902 


In terms of interpretation, the table below gives some general guidance.

| VIF Value     | Multicollinearity  | Interpretation                                                                     |
| ------------- | ------------------ | ---------------------------------------------------------------------------------- |
| 1             | None               | No issues and no action required.                                                  |
| 1 to 5        | Small to Moderate  | Some caution needed, but not usually a problem. Concern gets greater closer to 5.  |
| > 5           | High               | Standard errors will be larger than usual and inference will become unstable.      |
| > 10          | Severe             | Standard errors will have blown-up and inference will become untrustworthy.        |

In the example above, both `wt` and `wt.copy` have VIF > 10, showing *severe* multicollinearity. This points to a big problem with the model, specifically in relation to those two predictors. Notice that `hp` has 1 < VIF < 5, which gives little reason for concern. For `cyl`, that value is also 1 < VIF < 5, however it is much closer to 5 and this may be slightly more concerning. In this situation, we would likely remove either `wt` or `wt.copy`. If we do so and then recalculate VIF, we get

In [ ]:
mod <- lm(mpg ~ wt + hp + cyl, data=mtcars)
print(vif(mod))

      wt       hp      cyl 
2.580486 3.258481 4.757456 


So there is now no immediate cause for concern. The VIF for `cyl` is still getting close to 5, which is something we may need to bear in mind. This is likely due to the fact that engines that produce more horsepower often have more cylinders, which will also have an impact on the weight of the car. So this dependence is somewhat unavoidable, but could cause us to question whether `cyl` is adding anything useful to the model. 

[^NASA-foot]: [Faraway (2005)](https://www.utstat.toronto.edu/~brunner/books/LinearModelsWithR.pdf) provides a real-world example of why this is *not* good practise. This concerns the delay in the discovery of the hole in the Ozone layer due to NASA's automatic data analysis algorithms discarding very low readings assumed to be mistakes.

[^hatmat-foot]: This function is named after the fact that the leverage values come from the diagonal a special matrix called the *hat matrix*, which is used to form the predicted values (i.e. it puts the "hats" on $y$ to form $\hat{y}$).

[^levcutoff-foot]: You can verify this for yourself by also calculating `2 * mean(lev)`.